# 💫Training

### User의 질문을 아래 19가지의 의도로 분류하는 모델
#### Accuracy : 92~3%

In [ ]:
!pip3 install transformers==4.6.1
!pip install adamp

In [ ]:
#필요한 라이브러리 불러오기
import argparse
import os
import pickle
import random
import time
import warnings
from copy import deepcopy
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from adamp import AdamP
from sklearn.metrics import f1_score
from torch import nn, optim
from torch.cuda.amp import autocast, GradScaler
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from tqdm import tqdm
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, \
    ElectraForSequenceClassification, AdamW, get_cosine_with_hard_restarts_schedule_with_warmup

In [ ]:
GPU_NUM = 1 # 원하는 GPU 번호 입력
# device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# torch.cuda.set_device(device) # change allocation of current GPU
device

device(type='cuda')

In [ ]:
#random seed 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
seed_everything(42)

In [ ]:
#데이터를 불러오기 위한 함수
def load_data(file_path):
    with open(file_path, 'r') as f:
        data = [line.strip() for line in f.readlines()]
    return data

#전처리한 train 데이터 불러오기
# train_data = pd.read_csv("final.csv")
train_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/fashion_reader/question_intention_classification/new_data.csv")

In [ ]:
train_data.head()

,question,answer,label
0,상품 이름이 뭐야?,이름,0
1,모델이 입고 있는 옷 이름이 뭐야?,이름,0
2,이거 이름 알려줘,이름,0
3,상품 이름이 어떻게 돼?,이름,0
4,이름 좀 알려줘,이름,0


In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  306 non-null    object
 1   answer    306 non-null    object
 2   label     306 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


### NLP_Dataset 클래스 선언

In [ ]:
class NLP_Dataset(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataset = dataframe
        self.question = dataframe['question']
        self.labels = dataframe['label']
        self.tokenizer = tokenizer

    def __getitem__(self, idx) :
        tokenized_text = self.tokenizer(self.question[idx], 
                                        max_length=CFG.tokenizer_max_length, 
                                        padding='max_length', 
                                        truncation=True, 
                                        return_tensors='pt',
                                        add_special_tokens=True)
        
        tokenized_text['label'] = self.labels[idx]
        return tokenized_text

    def __len__(self):
        return len(self.labels)

### get_model 함수 선언
network, optimizer, scaler, scheduler, criterion 반환

In [ ]:
def get_model():
    network = AutoModelForSequenceClassification.from_pretrained(CFG.model_name, num_labels=CFG.n_classes, hidden_dropout_prob=CFG.dropout_rate).to("cuda")
    optimizer = AdamP(network.parameters(), lr=CFG.lr, betas=(0.9, 0.999), weight_decay=1e-2)
    scaler = GradScaler()
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)
    criterion = nn.CrossEntropyLoss().to("cuda")
    return network, optimizer, scaler, scheduler, criterion

### training_per_step 선언

In [ ]:
def training_per_step(model, loss_fn, optimizer, scaler, input_ids, attention_mask, labels, device):
    '''매 step마다 학습을 하는 함수'''
    model.train()
    
    with autocast():
        labels = labels.to(device)
        outputs = model(input_ids.to(device), attention_mask = attention_mask.to(device))[0]
        loss = loss_fn(outputs, labels)
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()

    return loss

### validating_per_steps 선언

In [ ]:
def validating_per_steps(epoch, model, loss_fn, test_loader, device):
    '''특정 step마다 검증을 하는 함수'''
    model.eval()

    loss_sum = 0
    sample_num = 0
    preds_all = []
    targets_all = []

    pbar = tqdm(test_loader, total=len(test_loader), position=0, leave=True)
    for batch in pbar :
        labels = batch.pop('label').to(device)
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)
        
        preds = model(input_ids, attention_mask = attention_mask)[0]
        
        preds_all += [torch.argmax(preds, 1).detach().cpu().numpy()]
        targets_all += [labels.detach().cpu().numpy()]

        loss = loss_fn(preds, labels)

        loss_sum += loss.item()*labels.shape[0]
        sample_num += labels.shape[0]

        description = f"epoch {epoch + 1} loss: {loss_sum/sample_num:.4f}"
        pbar.set_description(description)
    
    preds_all = np.concatenate(preds_all)
    targets_all = np.concatenate(targets_all)
    accuracy = (preds_all == targets_all).mean()
    f1 = f1_score(preds_all, targets_all, average='macro')
    print('validation multi-class accuracy = {:.4f}, f1 score = {:.4f}'.format(accuracy, f1))
    print(classification_report(targets_all, preds_all))
    return accuracy, f1

### Train 함수 선언

In [ ]:
def train(model, loss_fn, optimizer, scaler, train_loader, test_loader, scheduler, device):
    '''training과 validating을 진행하는 함수'''
    prev_acc = 0
    prev_f1 = 0
    global_steps = 0
    for epoch in range(CFG.epochs):
        running_loss = 0
        sample_num = 0
        preds_all = []
        targets_all = []
        
        pbar = tqdm(enumerate(train_loader), total=len(train_loader), position=0, leave=True)
        for step, batch in pbar:
            # training phase
            labels = batch.pop('label')
            input_ids = batch['input_ids'].squeeze(1)
            attention_mask = batch['attention_mask'].squeeze(1)
    
            loss = training_per_step(model, loss_fn, optimizer, scaler, input_ids.to(device), attention_mask.to(device), labels.to(device), device)
            running_loss += loss.item()*labels.shape[0]
            sample_num += labels.shape[0]
            
            global_steps += 1
            description = f"{epoch+1}epoch {global_steps: >4d}step | loss: {running_loss/sample_num: .4f} "
            pbar.set_description(description)

            if scheduler is not None :
                scheduler.step() 
                
            # validating phase
            if global_steps % CFG.logging_step == 0 :
                with torch.no_grad():
                    acc, f1 = validating_per_steps(epoch, model, loss_fn, test_loader, device)
                if f1 > prev_f1:
                    # torch.save(model, "./output/baseline.pt")
                    torch.save(model, "/content/drive/MyDrive/Colab Notebooks/fashion_reader/question_intention_classification/output/new_baseline.pt")
                    prev_f1 = f1
                    
                # wandb.log({
                # 'eval/acc' : acc,
                # 'eval/f1' : f1,
                # 'global_steps': global_steps
                # })
                print(f'eval/acc : {acc}, eval/f1 : {f1}, global_steps: {global_steps}')
                    
            else:
                print(f" global_steps : {global_steps}")
                # wandb.log({'global_steps':global_steps})  

### 학습을 위한 config 설정

In [ ]:
class CFG:
    tokenizer_max_length = 35
    batch_size = 32
#     model_name = "xlm-roberta-large" # 19,069 #93%
    model_name = "xlm-roberta-base" # 11,548 #93%
    # model_name = "bert-base-multilingual-cased" # 7,461 #93%
#     model_name = 'albert-base-v2' # 4,445 #23%
    lr = 0.000005
    epochs = 100
    change_mask_prop = 0.
    dropout_rate = 0
    n_classes = 19
    logging_step = 15

### tokenizer와 데이터 불러오기 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_data['question'], train_data['label'],
                                                    stratify= train_data['label'], 
                                                    test_size=0.2)

In [ ]:
train_data = pd.DataFrame(zip(X_train, y_train), columns=['question', 'label'])
valid_data = pd.DataFrame(zip(X_test, y_test), columns=['question', 'label'])

In [ ]:
train_data.label.value_counts()

10    21
4     17
14    17
15    16
16    14
8     14
17    13
6     13
5     13
9     12
11    12
3     12
1     10
2     10
18    10
7     10
12    10
13    10
0     10
Name: label, dtype: int64

In [ ]:
valid_data.label.value_counts()

10    6
15    4
14    4
4     4
5     4
17    4
18    3
7     3
1     3
3     3
6     3
9     3
8     3
11    3
16    3
0     3
12    2
13    2
2     2
Name: label, dtype: int64

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
train_set = NLP_Dataset(train_data, tokenizer)
valid_set = NLP_Dataset(valid_data, tokenizer)

Downloading:   0%|          | 0.00/512 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

In [ ]:
train_iter = DataLoader(train_set, batch_size=CFG.batch_size, shuffle=True)
val_iter = DataLoader(valid_set, batch_size=CFG.batch_size, shuffle=True)

In [ ]:
network, optimizer, scaler, scheduler, criterion = get_model()

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense

### wandb setting

In [ ]:
# import wandb
# os.environ['WANDB_LOG_MODEL'] = 'true'
# os.environ['WANDB_WATCH'] = 'all'
# os.environ['WANDB_SILENT'] = 'true'
# wandb.login()
# wandb.init(project='fashion_reader_question_classification', name='baseline_tjdnsgkek')

In [ ]:
train(network, criterion, optimizer, scaler, train_iter, val_iter, scheduler, "cuda")

1epoch    1step | loss:  2.9358 :  12%|█▎        | 1/8 [00:00<00:04,  1.54it/s]

 global_steps : 1


1epoch    2step | loss:  2.9475 :  25%|██▌       | 2/8 [00:00<00:02,  2.17it/s]

 global_steps : 2


1epoch    3step | loss:  2.9591 :  38%|███▊      | 3/8 [00:01<00:01,  2.58it/s]

...

 global_steps : 798


100epoch  799step | loss:  0.3089 :  88%|████████▊ | 7/8 [00:02<00:00,  2.66it/s]

 global_steps : 799


100epoch  800step | loss:  0.3077 : 100%|██████████| 8/8 [00:03<00:00,  2.67it/s]

 global_steps : 800


# 💌Inference

In [ ]:
class CFG:
    tokenizer_max_length = 35
    batch_size = 1
    model_name = "xlm-roberta-base"

In [ ]:
class NLP_Dataset_test(Dataset):
    def __init__(self, dataframe, tokenizer):
        self.dataset = dataframe
        self.question = dataframe['question']
        self.labels = dataframe['label']
        self.tokenizer = tokenizer

    def __getitem__(self, idx):
        tokenized_text = self.tokenizer(self.question[idx],
                                        max_length=CFG.tokenizer_max_length,
                                        padding='max_length',
                                        truncation=True,
                                        return_tensors='pt',
                                        add_special_tokens=True)
        
        tokenized_text['label'] = self.labels[idx]
        return tokenized_text

    def __len__(self):
        return len(self.labels)

In [ ]:
names = {0: '이름',
         1: '가격',
         2: '사이즈 옵션',
         3: '소재',
         4: '비침',
         5: '카테고리',
         6: '색 옵션',
         7: '두께감',
         8: '신축성',
         9: '촉감',
         10: '핏',
         11: '안감',
         12: '스타일',
         13: '프린팅',
         14: '상의 색',
         15: '하의 색',
         16: '상의 카테고리',
         17: '하의 카테고리',
         18: '넥 라인'}

### 학습된 모델 불러오기

In [ ]:
network = torch.load("/content/drive/MyDrive/Colab Notebooks/fashion_reader/question_intention_classification/output/new_baseline.pt") #기존까지 학습된 모델 load
pretrained_model_state = deepcopy(network.state_dict())
network.load_state_dict(pretrained_model_state)
device = "cuda"
network.to(device)
network.eval()

XLMRobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (Laye

In [ ]:
input_str = "옷이 체크야?"
test_df = pd.DataFrame(zip([input_str],[-1]), columns=['question', 'label'])
tokenizer = AutoTokenizer.from_pretrained(CFG.model_name)
test_set = NLP_Dataset_test(test_df, tokenizer)
test_loader = DataLoader(dataset=test_set, batch_size=CFG.batch_size, shuffle=False)

### Test Data로 Inference

In [ ]:
preds_all = []
prediction_array=[]
with tqdm(test_loader,
          total=test_loader.__len__(),
          unit='batch') as test_bar:
    for batch in test_bar:
        input_ids = batch['input_ids'].squeeze(1).to(device)
        attention_mask = batch['attention_mask'].squeeze(1).to(device)

        preds = network(input_ids, attention_mask = attention_mask)[0]
        preds_all += [torch.argmax(preds, 1).detach().cpu().numpy().item()]

100%|██████████| 1/1 [00:00<00:00,  3.98batch/s]


In [ ]:
print(f'유저 질문 : {input_str}')
print(f'질문 의도 : {preds_all[0]}, {names[preds_all[0]]}')

유저 질문 : 옷이 체크야?
질문 의도 : 13, 프린팅
